Notebook that demos how to reconcile 
- the csv exported from the saxo website
- vs TITRE CFD from Marketflow


Usage
- uncomment one of the input sets in the cell below and run the notebook

In [ ]:
"""
# input set 1: saxo vs mf lebanon
fn_mf = "mfxx_titre_cfd_20180629/mf-lb_titre_cfd_20180629.csv"
fn_saxo = "saxo21062018.csv"
fn_out = 'margin_recon-saxo-mf-lb-20180630.xlsx'

"""
# input set 2: saxo vs mf dubai
fn_mf = "mfxx_titre_cfd_20180629/mf-dxb_titre_cfd_20180629.csv"
fn_saxo = "saxo21062018.csv"
fn_out = 'margin_recon-saxo-mf-dxb-20180630.xlsx'


In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

## read mf and saxo files

In [ ]:
df_mf = pd.read_csv(fn_mf)
df_mf = df_mf[['TIT_COD', 'TIT_NOM', 'TIT_MAR_LN', ]] # 'TIT_MAR_SH', 'TIT_MAR_LN_MC', 'TIT_MAR_SH_MC', 'TIT_STY_COD'
df_mf.head()

In [ ]:
df_saxo = pd.read_csv(fn_saxo, encoding='latin1')
df_saxo = df_saxo.rename(columns={'Unnamed: 6': 'margin_pct'})
df_saxo = df_saxo[['Symbol', 'Description', 'margin_pct']] # , 'margin_desc'
df_saxo = df_saxo[pd.notnull(df_saxo['Symbol'])]
df_saxo['margin_pct'] = df_saxo['margin_pct'].apply(lambda x: float(x.replace('%','').replace('Automatic Execution', '100.00')))
df_saxo.head()

In [ ]:
# set(df_saxo['Margin Requirement']), set(df_saxo['margin_pct'])
set(df_saxo['margin_pct'])

## preprocess saxo file

In [ ]:
# generate symbol that is common to MF and Saxo
df_saxo['symbol_common'] = df_saxo['Symbol'].apply(lambda x: x.split(':')[0])
# rename columns to common nomenclature
df_saxo = df_saxo.rename(columns={'Description': 'name'})
# drop unnecessary column
del df_saxo['Symbol']


df_saxo.head()

## preprocess mflb

In [ ]:
# calculate symbol that is common with saxo
df_mf['symbol_common'] = df_mf['TIT_COD'].apply(lambda x: x.split(' ')[1])
# rename columns to common nomenclature
df_mf = df_mf.rename(columns={'TIT_NOM': 'name', 'TIT_MAR_LN': 'margin_pct'})
# drop unnecessary column
del df_mf['TIT_COD']

df_mf.head()

## merge saxo with mflb

In [ ]:
df_merged = df_saxo.merge(df_mf, on='symbol_common', how='outer', suffixes=['_saxo', '_mf'])
df_merged.head()

## split as "found"
- found in saxo but not in mf
- found in mf but not in saxo
- found in both

In [ ]:
df_insaxo_notmf = df_merged[pd.isnull(df_merged['name_mf'])]
df_insaxo_notmf.shape

In [ ]:
df_notsaxo_inmf = df_merged[pd.isnull(df_merged['name_saxo'])]
df_notsaxo_inmf.shape

In [ ]:
df_both = df_merged[(pd.notnull(df_merged['name_mf'])) & (pd.notnull(df_merged['name_saxo']))].copy()
df_both.shape

In [ ]:
df_saxo.shape[0], df_mf.shape[0], df_both.shape[0]

## in "both", check differences

In [ ]:
df_both.head()

In [ ]:
df_both['mf - saxo*1.25'] = abs(df_both['margin_pct_mf'] - df_both['margin_pct_saxo'] * 1.25)

In [ ]:
df_both['mf - saxo*1.25'].describe()

In [ ]:
df_both.sort_values(['mf - saxo*1.25'], ascending=False).reset_index()['mf - saxo*1.25'].plot()
plt.show()

In [ ]:
df_both.sort_values(['mf - saxo*1.25'], ascending=False).tail()

## save

In [ ]:
writer = pd.ExcelWriter(fn_out)
# df_insaxo_notmflb.to_excel(writer,'Sheet1')
df_notsaxo_inmf.to_excel(writer,'in mf but not saxo', index=False)
df_both.to_excel(writer,'in both', index=False)
writer.save()